# prediction based only on text part
applying TFIDF on titles and fuzzy match on image phash

# Load Libraries

In [ ]:
!pip install ../input/pandarallel151whl/pandarallel-1.5.1-py3-none-any.whl

In [ ]:
!pip install cupy

In [ ]:
import cudf

In [ ]:
import numpy as np, pandas as pd, gc
import cv2, matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
stemmer = SnowballStemmer('english')
import unicodedata
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from sklearn.preprocessing import normalize
TEST = True
CHUNK=1024*4
MAX_NUM_OF_MATCH = 60
WORD2VEC_VEC_SIZE = 100
TFIDF_THRESHOLD = 0.75
import scipy
preproc_txt = True
use_img_fuzzy=True
# from pandarallel import pandarallel
# pandarallel.initialize()
model_path = "/kaggle/input/sharpee-model-weight/"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load Data

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')
train = pd.read_csv('../input/shopee-product-matching/train.csv')
if TEST:
    df = train
else:
    df = test
    
if TEST:
    tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
    df['target'] = df.label_group.map(tmp)
    df.head(2)
print('Dataframe shape is',df.shape )

In [ ]:
import sys
sys.path.append('../input/timm-master/pytorch-image-models-master')
# sys.path.append(model_path)
import numpy as np 
import pandas as pd 

import math
import random 
import os 
import cv2

from tqdm import tqdm 

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

import torch 
from torch.utils.data import Dataset 
from torch import nn
import torch.nn.functional as F 

import gc
# import cudf
# import cuml
# import cupy
# from cuml.feature_extraction.text import TfidfVectorizer
# from cuml.neighbors import NearestNeighbors
import timm

In [ ]:
def get_avg_f1(targets,preds):
    def f1score(target,pred):
        n = len( np.intersect1d(target,pred) )
        return 2*n / (len(target)+len(pred))
    avg_f1_score = sum([f1score(*i) for i in zip(targets,preds)])/len(targets)
    return avg_f1_score

def combine_for_sub(cols):
    x = np.concatenate(cols)
    return ' '.join( np.unique(x) )

def combine_for_testing(cols):
    x = np.concatenate(cols)
    return np.unique(x)

def cleanData(dataParse):
    data = unicodedata.normalize('NFKC', dataParse)
    data = re.sub(r'【.*】', '', data)
    data = re.sub(r'\[.*\]', '', data)
    data = re.sub(r'「.*」', '', data)
    data = re.sub(r'\(.*\)', '', data)
    data = re.sub(r'\<.*\>', '', data)
    data = re.sub(r'[※@◎].*$', '', data)
    return data.lower() #Returns the parsed tweets


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(cleanData(text), pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            if token == 'xxxx':
                continue
            result.append(lemmatize_stemming(token))
    
    return " ".join(result)

def get_tfidf_pred(df,CHUNK = CHUNK,threshold = TFIDF_THRESHOLD,max_features = 25_000):
    print('Computing text embeddings...')

    model = TfidfVectorizer(stop_words='english', binary=True, max_features=max_features,norm="l2")
    if preproc_txt:
        text_embeddings = model.fit_transform(df.title.apply(preprocess))
    else:
        text_embeddings = model.fit_transform(df.title)
    preds = get_match_with_cosine_similarity(text_embeddings,df.posting_id,threshold,CHUNK)
    del model, text_embeddings
    return preds

def get_match_with_cosine_similarity(embeddings,posting_ids,threshold,CHUNK):
    if isinstance(embeddings,list):
        embeddings = np.vstack(embeddings)
        embeddings = normalize(embeddings,norm="l2")
    elif isinstance(embeddings,scipy.sparse.csr.csr_matrix):
        embeddings = embeddings.todense()
    elif isinstance(embeddings,pd.core.series.Series):
        embeddings =embeddings.values()
        
    else:
        Exception("Please add data handle method, embeding should be np matrix")

    
    embeddings = torch.from_numpy(embeddings)
    embeddings = embeddings.to(DEVICE)
    preds = []
    print('Finding similar images...')
    CTS = len(embeddings)//CHUNK
    if len(embeddings)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b, len(embeddings))
        print('chunk',a,'to',b)
        distances = torch.matmul(embeddings, embeddings[a:b].T).T
        distances = distances.data.cpu().numpy()
        for k in range(b-a):
#             import pdb;pdb.set_trace()
            idxs = np.where(distances[k,]>threshold)[0][:]
            if len(idxs)>MAX_NUM_OF_MATCH:
                idxs = distances[k,].argsort()[-MAX_NUM_OF_MATCH:][::-1]
#                 print(f"Found {str(len(idxs))}, and it is regulated to MAX_NUM_OF_MATCH {str(MAX_NUM_OF_MATCH)}")
            o = pd.Series(posting_ids).iloc[idxs].values
            preds.append(o)
    del embeddings
    return preds


In [ ]:
class CFG:
    
    img_size = 512
    batch_size = 12
    seed = 2020
    
    device = DEVICE
    classes = 11014
    
    scale = 30 
    margin = 0.5



def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)


def get_test_transforms():

    return A.Compose(
        [
            A.Resize(CFG.img_size,CFG.img_size,always_apply=True),
            A.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=DEVICE)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = None,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))
#         import pdb;pdb.set_trace()
        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnext50_32x4d':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = ArcMarginProduct(
            final_in_features,
            n_classes,
            scale = scale,
            margin = margin,
            easy_margin = False,
            ls_eps = 0.0
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2) 

        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 


class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)


def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

def get_model(model_name = None, model_path = None, n_classes = None):
    
    model = ShopeeModel(model_name = model_name)
    if model_name == 'eca_nfnet_l0':
        model = replace_activations(model, torch.nn.SiLU, Mish())
    model.eval()
    model.load_state_dict(torch.load(model_path,map_location=torch.device(DEVICE)))
    model = model.to(CFG.device)
    
    return model 

class EnsembleModel(nn.Module):
    
    def __init__(self):
        super(EnsembleModel,self).__init__()
        
        self.m1 = get_model('eca_nfnet_l0',f'{model_path}arcface_512x512_nfnet_l0 (mish).pt')
        self.m2 = get_model('tf_efficientnet_b5_ns',f'{model_path}arcface_512x512_eff_b5_.pt')
        
    def forward(self,img,label):
        
        feat1 = self.m1(img,label)
        feat2 = self.m2(img,label)
    
        return (feat1 + feat2) / 2

def get_image_embeddings(image_paths, model_name = None, model_path = None):
    embeds = []
    
    model = EnsembleModel()
    
    image_dataset = ShopeeDataset(image_paths=image_paths,transforms=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.batch_size,
        pin_memory=True,
        drop_last=False,
        num_workers=4
    )
    
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            if DEVICE.type!="cpu":
                img = img.cuda()
                label = label.cuda()
            feat = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

if TEST:
    df = pd.read_csv('../input/shopee-product-matching/train.csv')
    image_paths = '../input/shopee-product-matching/train_images/' + df['image']
else:
    df = pd.read_csv('../input/shopee-product-matching/test.csv')
    image_paths = '../input/shopee-product-matching/test_images/' + df['image']
# df.head()

image_embeddings = get_image_embeddings(image_paths.values)
df["image_pred"] = get_match_with_cosine_similarity(image_embeddings,df.posting_id,threshold=0.36,CHUNK=CHUNK)

In [ ]:
def hamming_distance(phash1, phash2):
    '''
    helper function to calculate phash similarity
    '''
    phash1 = bin(int(phash1, 16))[2:].zfill(64)
    phash2 = bin(int(phash2, 16))[2:].zfill(64)
    distance = np.sum([i != j for i, j in zip(phash1, phash2)])
    return distance
def hamming_distance_bin(phash1, phash2,thrshold = 5):
    '''
    helper function to calculate phash similarity
    '''
    count = 0
    for i,j in zip(phash1, phash2):
        if i != j:
            count+=1
        if count>thrshold:
            return False
    return True

def get_match_by_fuzzy_img_phase(phase,phases,posting_ids,thrshold = 5):
    return posting_ids[phases.applymap(lambda x:hamming_distance_bin(x,phase,thrshold))].tolist()

In [ ]:
def get_tfidf_pred(df,CHUNK = CHUNK,threshold = TFIDF_THRESHOLD,max_features = 25_000):
    print('Computing text embeddings...')

    model = TfidfVectorizer(stop_words='english', binary=True,analyzer="char",ngram_range =(10,11) ,max_features=max_features,norm="l2")
#     if preproc_txt:
#         text_embeddings = model.fit_transform(df.title.apply(preprocess))
#     else:
#     import pdb;pdb.set_trace()
    text_embeddings = model.fit_transform(df.image_phash)
    preds = get_match_with_cosine_similarity(text_embeddings,df.posting_id,threshold,CHUNK)
    del model, text_embeddings
    return preds

def get_match_with_cosine_similarity(embeddings,posting_ids,threshold,CHUNK):
    if isinstance(embeddings,list):
        embeddings = np.vstack(embeddings)
        embeddings = normalize(embeddings,norm="l2")
    elif isinstance(embeddings,scipy.sparse.csr.csr_matrix):
        embeddings = embeddings.todense()
    elif isinstance(embeddings,pd.core.series.Series):
        embeddings =embeddings.values()
        
    else:
        Exception("Please add data handle method, embeding should be np matrix")

    
    embeddings = torch.from_numpy(embeddings)
    embeddings = embeddings.to(DEVICE)
    preds = []
    print('Finding similar images...')
    CTS = len(embeddings)//CHUNK
    if len(embeddings)%CHUNK!=0: CTS += 1
    for j in range( CTS ):

        a = j*CHUNK
        b = (j+1)*CHUNK
        b = min(b, len(embeddings))
        print('chunk',a,'to',b)
        distances = torch.matmul(embeddings, embeddings[a:b].T).T
        distances = distances.data.cpu().numpy()
        for k in range(b-a):
#             import pdb;pdb.set_trace()
            idxs = np.where(distances[k,]>threshold)[0][:]
            if len(idxs)>MAX_NUM_OF_MATCH:
                idxs = distances[k,].argsort()[-MAX_NUM_OF_MATCH:][::-1]
#                 print(f"Found {str(len(idxs))}, and it is regulated to MAX_NUM_OF_MATCH {str(MAX_NUM_OF_MATCH)}")
            o = pd.Series(posting_ids).iloc[idxs].values
            preds.append(o)
    del embeddings
    return preds



To prevent memory errors, we will find similar titles in chunks. To faciliate this, we will use cosine similarity between text embeddings instead of KNN.

In [ ]:
%%time
# df['img_phase'] =  df.image_phash.parallel_apply(lambda x:get_match_by_fuzzy_img_phase(x,df.image_phash,df.posting_id))
df['tfidf'] = get_tfidf_pred(df)
# if TEST:
print(f"F1 score with tfidf: {str(get_avg_f1(df.target,df.tfidf))}")
# print(f"F1 score with iamge phase: {str(get_avg_f1(df.target,df.img_phase))}")
# df["matches"] = df[["tfidf","img_phase"]].apply(combine_for_testing,axis=1)
# print('CV Score =', get_avg_f1(df["matches"],df.target) )

In [ ]:
df.groupby("posting_id")["image_phash"].apply(lambda x:"/n".join(x))

# Write Submission CSV
In this notebook, the submission file below looks funny containing train information. But when we submit this notebook, the size of `test.csv` dataframe will be longer than 3 rows and the variable `COMPUTE_CV` will subsequently set to `False`. Then our submission notebook will compute the correct matches using the real test dataset and our submission csv for LB will be ok.

In [ ]:
if TEST:
    df["matches"] = df[["tfidf","img_phase"]].apply(combine_for_testing,axis=1)
    print('CV Score =', get_avg_f1(df["matches"],df.target) )
else:
    df['matches'] = df[["tfidf","img_phase","image_pred"]].apply(combine_for_sub,axis=1)
test = df
test[['posting_id','matches']].to_csv('submission.csv',index=False)
# sub.head()